# LCT 3 APM

In [12]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
import statsmodels.api as sm
import scipy as sp
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import statsmodels.stats.api as sms
import statsmodels.stats.proportion as ssp
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir(r"C:\Users\willi\OneDrive\Documents\Woolf Institute Data Analytics\Module 4 Advanced Predictive Modelling\04_ Live Class Data Sets")

In [2]:
# Import Data
bankloan = pd.read_csv("BANK LOAN.csv")
print("✅ Dataset Loaded Successfully")
print(bankloan.info())
print(bankloan.describe())

✅ Dataset Loaded Successfully
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SN         700 non-null    int64  
 1   AGE        700 non-null    int64  
 2   EMPLOY     700 non-null    int64  
 3   ADDRESS    700 non-null    int64  
 4   DEBTINC    700 non-null    float64
 5   CREDDEBT   700 non-null    float64
 6   OTHDEBT    700 non-null    float64
 7   DEFAULTER  700 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 43.9 KB
None
              SN         AGE      EMPLOY     ADDRESS     DEBTINC    CREDDEBT  \
count  700.00000  700.000000  700.000000  700.000000  700.000000  700.000000   
mean   350.50000    1.902857    8.388571    8.278571   10.260571    1.553457   
std    202.21688    0.765301    6.658039    6.824877    6.827234    2.117209   
min      1.00000    1.000000    0.000000    0.000000    0.400000    0.010000   
25%    175

In [3]:
# Change "AGE" variable to categorical

bankloan['AGE']=bankloan['AGE'].astype('category')

bankloan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   SN         700 non-null    int64   
 1   AGE        700 non-null    category
 2   EMPLOY     700 non-null    int64   
 3   ADDRESS    700 non-null    int64   
 4   DEBTINC    700 non-null    float64 
 5   CREDDEBT   700 non-null    float64 
 6   OTHDEBT    700 non-null    float64 
 7   DEFAULTER  700 non-null    int64   
dtypes: category(1), float64(3), int64(4)
memory usage: 39.2 KB


In [4]:
# Logisticc regression using Logit function

riskmodel = smf.logit(formula = 'DEFAULTER ~ AGE + EMPLOY + ADDRESS + DEBTINC + CREDDEBT + OTHDEBT', data = bankloan).fit()
print(riskmodel.summary())

Optimization terminated successfully.
         Current function value: 0.395291
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              DEFAULTER   No. Observations:                  700
Model:                          Logit   Df Residuals:                      692
Method:                           MLE   Df Model:                            7
Date:                Sat, 10 May 2025   Pseudo R-squ.:                  0.3120
Time:                        09:23:07   Log-Likelihood:                -276.70
converged:                       True   LL-Null:                       -402.18
Covariance Type:            nonrobust   LLR p-value:                 1.733e-50
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7882      0.264     -2.985      0.003      -1.306      -0.271
AGE[T.2]       0.2520      0.

In [5]:
# Re-run model with finalized variables
riskmodel = smf.logit(formula = 'DEFAULTER ~  EMPLOY + ADDRESS + DEBTINC + CREDDEBT', data = bankloan).fit()
print(riskmodel.summary())

Optimization terminated successfully.
         Current function value: 0.397671
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              DEFAULTER   No. Observations:                  700
Model:                          Logit   Df Residuals:                      695
Method:                           MLE   Df Model:                            4
Date:                Sat, 10 May 2025   Pseudo R-squ.:                  0.3079
Time:                        09:24:33   Log-Likelihood:                -278.37
converged:                       True   LL-Null:                       -402.18
Covariance Type:            nonrobust   LLR p-value:                 2.114e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7911      0.252     -3.145      0.002      -1.284      -0.298
EMPLOY        -0.2426      0.

Cross Validation

In [15]:
# Split the data into training and test sets

X_train, X_test = train_test_split(bankloan, test_size=0.3, random_state=42)
print("✅ Data Split into Training and Test Sets")


✅ Data Split into Training and Test Sets


In [11]:
print("X-Train Shape:",X_train.shape)


X-Train Shape: (490, 8)


In [10]:
print("X-Test Shape:",X_test.shape)

X-Test Shape: (210, 8)


Generate Classification Report

In [13]:
# Confusion Matrix

riskmodel=smf.logit(formula = 'DEFAULTER ~  EMPLOY + ADDRESS + DEBTINC + CREDDEBT', data = X_train).fit()

predicted_values1=riskmodel.predict()
threshold=0.3
predicted_class1=np.zeros(predicted_values1.shape)
predicted_class1[predicted_values1>threshold]=1

from sklearn.metrics import classification_report
print(classification_report(X_train['DEFAULTER'],predicted_class1))


Optimization terminated successfully.
         Current function value: 0.385825
         Iterations 7
              precision    recall  f1-score   support

           0       0.90      0.81      0.86       370
           1       0.56      0.73      0.64       120

    accuracy                           0.79       490
   macro avg       0.73      0.77      0.75       490
weighted avg       0.82      0.79      0.80       490



In [16]:
# Classification Report for Test data

predicted_values1=riskmodel.predict(X_test)
threshold=0.3
predicted_class1=np.zeros(predicted_values1.shape)
predicted_class1[predicted_values1>threshold]=1

print(classification_report(X_test['DEFAULTER'],predicted_class1))


              precision    recall  f1-score   support

           0       0.93      0.86      0.89       161
           1       0.62      0.78      0.69        49

    accuracy                           0.84       210
   macro avg       0.77      0.82      0.79       210
weighted avg       0.86      0.84      0.84       210



In [18]:
# K-Fold Cross Validation
from sklearn import linear_model
lmreg = linear_model.LogisticRegression()

y=bankloan.DEFAULTER
X=bankloan[['EMPLOY', 'ADDRESS', 'DEBTINC', 'CREDDEBT']]

from sklearn.model_selection import cross_val_predict
# from sklearn.metrics.classification import cohen_kappa_score

predicted_prob = cross_val_predict(lmreg, X, y, cv=4, method='predict_proba') 
threshold=0.3
predicted = predicted_prob[:,1]
predicted_class1=np.zeros(predicted.shape)
predicted_class1[predicted>threshold]=1
print(classification_report(y,predicted_class1))


              precision    recall  f1-score   support

           0       0.90      0.80      0.85       517
           1       0.57      0.75      0.65       183

    accuracy                           0.79       700
   macro avg       0.74      0.78      0.75       700
weighted avg       0.82      0.79      0.80       700

